## Guided BackPropagation: Практика

Как мы с вами увидели в теории, Guided Backpropagation — это не только самостоятельный метод для решения задачи интерпретации, но и составная часть других методов.

Guided Backpropagation является модификацией обычного алгоритма Vanilla Gradients. Главное отличие — в нём акцент идет только на те градиенты, что были больше нуля.  пропускаются только через положительные активации, что позволяет получить более четкие и интерпретируемые визуализации.

В этой практике рассмотрим, как применить метод Guided Backpropagation к глубокой нейронной сети для визуализации влияния различных входных признаков на выходное предсказание.

**Цели практики:**

- Понять основные концепции метода Guided Backpropagation
- Научиться применять этот метод на практике
- Самостоятельно реализовать класс Guided_backprop для моделей ResNet и AlexNet

<a href="https://ibb.co/gj69gg0"><img src="https://i.ibb.co/dW7MKKN/tim-foster-w-X64-Gjbclg-unsplash.jpg" alt="tim-foster-w-X64-Gjbclg-unsplash" border="0"></a>

**Quiz 1.**Начнем с вспоминания пройденного. Ответьте — что такое Hook в pyTorch?

**Ваш ответ:**


*   Механизм для регуляризация градиентов
*   Механизм, который позволяет пользователям встраивать свои функции в процесс работы модели
* Способ для ускорения обучения
* Способ для нормализации функции потерь




Хуки понадобятся нам для реализации Guided backpropagation. Создадим шаблон класса, который будет его осуществлять.



```
class Guided_backprop():

    def __init__(self, model):
        self.model = model
        self.image_reconstruction = None # Здесь будет итоговая карта активаций
        self.activation_maps = []  # сюда будем записывать f1, f2, ...
        self.model.eval()
        self.register_hooks()

    def register_hooks(self):
        def first_layer_hook_fn(module, grad_in, grad_out): # здесь будет функция для перехвата первого слоя
            pass

        def forward_hook_fn(module, input, output): # здесь будет функция для forward hook
            pass

        def backward_hook_fn(module, grad_in, grad_out): # здесь будет функция для backward hook
            pass


    def visualize(self, input_image, target_class):

        model_output = self.model(input_image)
        pass
```



Загрузим изображение с которым будем работать. Для интереса, рассмотрим пример, на котором находятся объекты сразу двух классов.

In [ ]:
# импортируем необходимые библиотеки
import torch
import requests
import numpy as np
from io import BytesIO
from torch import nn
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
# Загрузка изображения
url = 'https://github.com/aiedu-courses/all_datasets/blob/main/images/cat_and_dog.jpg?raw=true'

image_bytes = requests.get(url).content
image = Image.open(BytesIO(image_bytes)) # Снова рассмотрим конкретный пример x_0

# Преобразуем изображение для подачи обученной сети
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    # Ваш код здесь
])


tensor = transform(image).unsqueeze(0).requires_grad_()

**Quiz 2** Закончите функцию `transform`. Добавьте нормализацию (`transforms.Normalize`) с классическими средним и стандартным отклонением для ImageNet. В ответ укажите значение с координатами `tensor[0, 1, 1, 1]`  

In [ ]:
# Ваш код здесь

Теперь загрузим модель. Будем работать с resnet50.


In [ ]:
model = models.resnet50(pretrained=True)
model.eval();

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


**Quiz 3** Примените модель к изображению. Какой класс она спрогнозировала? В ответ укажите номер класса.

In [ ]:
# Ваш код здесь

tensor(179)

- Начнем дополнять класс для реализации Guided backpropagation. Начнем с `forward_hook_fn(module, input, output)`. Данный хук нужен нам, чтобы хранить карты, *получающиеся на определенных слоях* при прямом проходе.

**Quiz 4** Согласно этой логике, выберите правильную реализацию

```
`forward_hook_fn` на степик и запишите её в шаблон класса.

 def forward_hook_fn(module, input, output):
            # Выберите функцию на степик
```




- Теперь подумаем над `backward hook`.

**Quiz 5** Смотря на реализованный код, расставьте на платформе правильный порядок шагов для него.


```
def backward_hook_fn(module, grad_in, grad_out):
            grad = self.activation_maps.pop()
            grad[grad > 0] = 1
            
            positive_grad_out = torch.clamp(grad_out[0], min=0.0)
            new_grad_in = positive_grad_out * grad

            return (new_grad_in,)
```




- Также нам понадобится сохранить карту признаков изображения ($f^0$). Обычно рассматривается входное изображение, которое можно определить как input при forward для первого слоя модели, то есть:


```
def first_layer_hook_fn(module, grad_in, grad_out):
            self.image_reconstruction = grad_in[0]
```



**Отлично! Теперь реализуем все преобразования, которые наш Guided Backpropagation должен выполнять.**

Во-первых, нам необходимо извлечь все составляющие модели.

`modules = list(self.model.named_modules())`

Далее, мы будем применять forward и backward hook от конца к началу. Для всех преобразований изображения, кроме первого, мы будем регистрировать forward и backward хуки только после *определенных* слоев. Помните, после каких (ответ етсь в теории на степик)?

Хуки на первом слое снимем только после прохода по слоям предшествующим.

**Quiz 6**
Проверку соответствия слоя определенному типу можно осуществить функция `isinstance(object, type)`. Что должно находиться на месте типа для нашего случая?

Выберите ответ на степик и дополните код.

```
for name, module in modules:
  if isinstance(module, # Ваш ответ здесь):
    module.register_forward_hook(forward_hook_fn)
    module.register_backward_hook(backward_hook_fn)
```



        ##RESNET

        if 'resnet' in identify_model(self.model).lower():
          first_layer = modules[1][1]
          first_layer.register_backward_hook(first_layer_hook_fn)

        ###ALEXNET

        if 'alexnet' in identify_model(self.model).lower():
          first_layer = modules[1][1][0]
          first_layer.register_backward_hook(first_layer_hook_fn)

**На этом шаге реализована основвная часть Guided Backpropagation. Осталось добавить возможность визуализировать прогноз и готово!**

Визуализацию опишем по такой логике:

```

 def visualize(self, input_image, target_class):


        model_output = self.model(input_image) # прогнозируем класс
        self.model.zero_grad() # обнуляем градиенты
        pred_class = model_output.argmax().item() # извлекаем прогноз модели

        # делаем заготовку из 0 и 1, чтобы совершить backward pass по параметрам интересующего нас изображения

        grad_target_map = torch.zeros(model_output.shape,
                                      dtype=torch.float)
        if target_class is not None:
            grad_target_map[0][target_class] = 1
        else:
            grad_target_map[0][pred_class] = 1

        model_output.backward(grad_target_map)

        result = self.image_reconstruction.data[0].permute(1,2,0) # готовим результат к визуализации
        return result.numpy()
```

1. Во-первых, добавим возможность строить Guided backpropagation от любого интересующего нас класса. Для этого функция будет принимать аргумент `target class`. По умолчанию Guided backpropagation будем строить для спрогнозированного класса.
2. Во-вторых, все результаты будем возвращаться в виде numpy array.

**Теперь соберем все воедино и закончим класс.**


In [ ]:
import torch
from torch import nn
from torchvision import models, transforms
from PIL import Image
import matplotlib.pyplot as plt



def identify_model(model):
    return model.__class__.__name__


class Guided_backprop():
    def __init__(self, model):
        self.model = model
        self.image_reconstruction = None # Здесь будет итоговая карта активаций
        self.activation_maps = []  # сюда будем записывать f1, f2, ...
        self.model.eval()
        self.register_hooks()

    def register_hooks(self):
        def first_layer_hook_fn(module, grad_in, grad_out):
            self.image_reconstruction = grad_in[0]

        def forward_hook_fn(module, input, output):
            self.activation_maps.append(output)

        def backward_hook_fn(module, grad_in, grad_out):
            grad = self.activation_maps.pop() # извлекаем последнюю карту в списке (f_l, f_l-1, f_l-2...)

            # логическая функция при forward pass, после ReLU
            # если выходное значение не было равно 0, то делаем его единицей
            # и нулём иначе if the output value is positive, we set the value to 1,
            # and if the output value is negative, we set it to 0.
            grad[grad > 0] = 1

            #grad_out[0] будем записывать градиенты для каждой карты признаков
            # только если градиентны были больше нуля
            positive_grad_out = torch.clamp(grad_out[0], min=0.0)

            #Логическое И над результатами (эквивалетно умножению)
            new_grad_in = positive_grad_out * grad

            return (new_grad_in,)


        # AlexNet model
        modules = list(self.model.named_modules())

        # двигаемся по модулям, извлекам карты при forward и backward
        # для ReLU
        for name, module in modules:
            if isinstance(module, nn.ReLU):
                module.register_forward_hook(forward_hook_fn)
                module.register_backward_hook(backward_hook_fn)

        ##RESNET

        if 'resnet' in identify_model(self.model).lower():
          first_layer = modules[1][1]
          first_layer.register_backward_hook(first_layer_hook_fn)

        ###ALEXNET

        if 'alexnet' in identify_model(self.model).lower():
          first_layer = modules[1][1][0]
          first_layer.register_backward_hook(first_layer_hook_fn)


    def visualize(self, input_image, target_class):


        model_output = self.model(input_image) # прогнозируем класс
        self.model.zero_grad() # обнуляем градиентны
        pred_class = model_output.argmax().item() # извлекаем метку класса

        # делаем заготовку из 0 и 1, чтобы совершить backward pass по параметрам интересующего нас изображения
        grad_target_map = torch.zeros(model_output.shape,
                                      dtype=torch.float)
        if target_class is not None:
            grad_target_map[0][target_class] = 1
        else:
            grad_target_map[0][pred_class] = 1

        model_output.backward(grad_target_map)

        result = self.image_reconstruction.data[0].permute(1,2,0) # готовим результат к визуализации
        return result.numpy()

def normalize(image):
    "Функция для улучшения читаемости построенной карты"
    norm = (image - image.mean())/image.std()
    norm = norm * 0.1
    norm = norm + 0.5
    norm = norm.clip(0, 1)
    return norm

In [ ]:
guided_bp = Guided_backprop(model)
result = guided_bp.visualize(tensor, None)

result = normalize(result)
plt.imshow(result)
plt.show()

Постройте карту по классу tiger cat (282). Изменилась ли она?

In [ ]:
guided_bp = Guided_backprop(model)
result1 = guided_bp.visualize(tensor, 282)

result1 = normalize(result)
plt.imshow(result1)
plt.show()

Постройте Guided backprop по любому случайному классу. Сильно ли меняется результат?

In [ ]:
guided_bp = Guided_backprop(model)
result = guided_bp.visualize(tensor, 100)

result = normalize(result)
plt.imshow(result)
plt.show()